In [3]:
#import sys
#!conda install --yes --prefix {sys.prefix} torchvision

Solving environment: done

# All requested packages already installed.



In [59]:
#import sys
#!conda install --yes --prefix {sys.prefix} -c rdkit rdkit

Solving environment: done

## Package Plan ##

  environment location: /home/jgmeyer2/anaconda3

  added / updated specs: 
    - rdkit


The following NEW packages will be INSTALLED:

    libboost: 1.65.1-habcd387_4               
    py-boost: 1.65.1-py36hf484d3e_4           
    rdkit:    2018.03.4.0-py36h71b666b_1 rdkit

Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [1]:
import argparse
import os
import numpy as np
import math
import sys
import glob

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch.autograd as autograd
import torch

### Download images from SMILES

In [3]:
#print(os.listdir())
tables_list=glob.glob("downloaded_zinc_n2/*.txt")
print(tables_list)
print(tables_list[0])

['downloaded_zinc_n2/IACA.txt', 'downloaded_zinc_n2/JACB.txt', 'downloaded_zinc_n2/KACB.txt', 'downloaded_zinc_n2/JAAA.txt', 'downloaded_zinc_n2/KAAA.txt', 'downloaded_zinc_n2/IABB.txt', 'downloaded_zinc_n2/IABA.txt', 'downloaded_zinc_n2/KAEB.txt', 'downloaded_zinc_n2/IAAD.txt', 'downloaded_zinc_n2/JABA.txt', 'downloaded_zinc_n2/KAAB.txt', 'downloaded_zinc_n2/IACD.txt', 'downloaded_zinc_n2/JAEB.txt', 'downloaded_zinc_n2/JABD.txt', 'downloaded_zinc_n2/KAEA.txt', 'downloaded_zinc_n2/KAAD.txt', 'downloaded_zinc_n2/JACA.txt', 'downloaded_zinc_n2/IAAB.txt', 'downloaded_zinc_n2/JAEA.txt', 'downloaded_zinc_n2/JABB.txt', 'downloaded_zinc_n2/IABD.txt', 'downloaded_zinc_n2/KABA.txt', 'downloaded_zinc_n2/JAAB.txt', 'downloaded_zinc_n2/IAAA.txt', 'downloaded_zinc_n2/JACD.txt', 'downloaded_zinc_n2/KABB.txt', 'downloaded_zinc_n2/IAED.txt', 'downloaded_zinc_n2/JAAD.txt', 'downloaded_zinc_n2/IACB.txt', 'downloaded_zinc_n2/JAED.txt', 'downloaded_zinc_n2/IAEA.txt', 'downloaded_zinc_n2/KACA.txt', 'downlo

Loop through the tables found above, take the first column containing smiles text, and append to master list

In [4]:
import csv
    
smileslist = []
        
for table in tables_list:
    with open(table) as inf:
        reader = csv.reader(inf, delimiter="\t")
        smiles = list(zip(*reader))[0]
        smileslist.extend(list(smiles)[1:])
        print(table)
    
print(len(smileslist))

downloaded_zinc_n2/IACA.txt
downloaded_zinc_n2/JACB.txt
downloaded_zinc_n2/KACB.txt
downloaded_zinc_n2/JAAA.txt
downloaded_zinc_n2/KAAA.txt
downloaded_zinc_n2/IABB.txt
downloaded_zinc_n2/IABA.txt
downloaded_zinc_n2/KAEB.txt
downloaded_zinc_n2/IAAD.txt
downloaded_zinc_n2/JABA.txt
downloaded_zinc_n2/KAAB.txt
downloaded_zinc_n2/IACD.txt
downloaded_zinc_n2/JAEB.txt
downloaded_zinc_n2/JABD.txt
downloaded_zinc_n2/KAEA.txt
downloaded_zinc_n2/KAAD.txt
downloaded_zinc_n2/JACA.txt
downloaded_zinc_n2/IAAB.txt
downloaded_zinc_n2/JAEA.txt
downloaded_zinc_n2/JABB.txt
downloaded_zinc_n2/IABD.txt
downloaded_zinc_n2/KABA.txt
downloaded_zinc_n2/JAAB.txt
downloaded_zinc_n2/IAAA.txt
downloaded_zinc_n2/JACD.txt
downloaded_zinc_n2/KABB.txt
downloaded_zinc_n2/IAED.txt
downloaded_zinc_n2/JAAD.txt
downloaded_zinc_n2/IACB.txt
downloaded_zinc_n2/JAED.txt
downloaded_zinc_n2/IAEA.txt
downloaded_zinc_n2/KACA.txt
downloaded_zinc_n2/IAEB.txt
57885


In [6]:
#### cut the first line from the list which is the header
smiles= smileslist[1:len(smileslist)]
#print(len(smileslist))
#print(len(smiles))

In [7]:
from rdkit import Chem
from rdkit.Chem import Draw

import matplotlib.pyplot as plt
%matplotlib inline

i=1
for x in smiles:
    x_mol = Chem.MolFromSmiles(str(x))
    Draw.MolToFile(x_mol, fileName="./molecules100/"+str(i)+"a.png", size=(100, 100), kekulize=True, wedgeBonds=True, imageType="png")
    i+=1
    #urllib.request.urlretrieve(url_pre+str(x)+url_post, out_pre+str(x)+out_suffix)

### Part to use my own images

In [8]:
import glob
import random
import os
import numpy as np

from torch.utils.data import Dataset
from PIL import Image
import torchvision.transforms as transforms

class ImageDataset(Dataset):
    def __init__(self, folder_path, transforms_=None):
        self.transform = transforms.Compose(transforms_)
        self.files = sorted(glob.glob('%s/*.*' % folder_path))

    def __getitem__(self, index):
        img = Image.open(self.files[index % len(self.files)])
        img = self.transform(img)
        return img

    def __len__(self):
        return len(self.files)

In [9]:
img_shape = (3, 100, 100)

cuda = True if torch.cuda.is_available() else False
print(cuda)

True


## Option Settings

In [11]:
n_epochs=200 #number of epochs of training
batch_size=64 #size of the batches
lr=0.0002 #adam: learning rate
b1=0.5  #"adam: decay of first order momentum of gradient")
b2=0.999 #adam: decay of first order momentum of gradient")
n_cpu=8
latent_dim=50
img_size=100
channels=3
n_critic=5
clip_value=0.01
sample_interval=815
img_shape = (channels, img_size, img_size)

### nn.modules

In [37]:

folder_path = "./molecules100"
transforms_ = [ transforms.Resize(50),
                transforms.CenterCrop(50),
                transforms.ToTensor(),
                transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5)) ]
dataloader = DataLoader(ImageDataset(folder_path, transforms_=transforms_),
                        batch_size=100, shuffle=True, num_workers=8)

In [38]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *block(100, 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, int(np.prod(img_shape))),
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        img = img.view(img.shape[0], *img_shape)
        return img

In [49]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(int(np.prod(img_shape)), 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
        )

    def forward(self, img):
        img_flat = img.view(img.shape[0], -1)
        validity = self.model(img_flat)
        return validity

In [50]:
# Loss weight for gradient penalty
lambda_gp = 10

# Initialize generator and discriminator
generator = Generator()
discriminator = Discriminator()


In [51]:
if cuda:
    generator.cuda()
    discriminator.cuda()

Determine optimal learning rate

# Actual training now

In [78]:
# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor


def compute_gradient_penalty(D, real_samples, fake_samples):
    """Calculates the gradient penalty loss for WGAN GP"""
    # Random weight term for interpolation between real and fake samples
    alpha = Tensor(np.random.random((real_samples.size(0), 1, 1, 1)))
    # Get random interpolation between real and fake samples
    interpolates = (alpha * real_samples + ((1 - alpha) * fake_samples)).requires_grad_(True)
    d_interpolates = D(interpolates)
    fake = Variable(Tensor(real_samples.shape[0], 1).fill_(1.0), requires_grad=False)
    # Get gradient w.r.t. interpolates
    gradients = autograd.grad(
        outputs=d_interpolates,
        inputs=interpolates,
        grad_outputs=fake,
        create_graph=True,
        retain_graph=True,
        only_inputs=True,
    )[0]
    gradients = gradients.view(gradients.size(0), -1)
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    return gradient_penalty



In [79]:
batches_done = 0
for epoch in range(1000):
    for i, imgs in enumerate(dataloader):

        # Configure input
        real_imgs = Variable(imgs.type(Tensor))

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Sample noise as generator input
        z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], 100))))

        # Generate a batch of images
        fake_imgs = generator(z)

        # Real images
        real_validity = discriminator(real_imgs)
        # Fake images
        fake_validity = discriminator(fake_imgs)
        # Gradient penalty
        gradient_penalty = compute_gradient_penalty(discriminator, real_imgs.data, fake_imgs.data)
        # Adversarial loss
        d_loss = -torch.mean(real_validity) + torch.mean(fake_validity) + lambda_gp * gradient_penalty

        d_loss.backward()
        optimizer_D.step()

        optimizer_G.zero_grad()

        # Train the generator every n_critic steps
        if i % 5 == 0:

            # -----------------
            #  Train Generator
            # -----------------

            # Generate a batch of images
            fake_imgs = generator(z)
            # Loss measures generator's ability to fool the discriminator
            # Train on fake images
            fake_validity = discriminator(fake_imgs)
            g_loss = -torch.mean(fake_validity)

            g_loss.backward()
            optimizer_G.step()

            print(
                "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
                % (epoch, 200, i, len(dataloader), d_loss.item(), g_loss.item())
            )
            
            
            if batches_done % 81500 == 0:
                save_image(fake_imgs.data[:25], "images100_b1_0p5_lr_0p00002n3/%d.png" % batches_done, nrow=5, normalize=True)

            batches_done += 5

[Epoch 0/200] [Batch 0/815] [D loss: -5.789902] [G loss: -23.082102]
[Epoch 0/200] [Batch 5/815] [D loss: -3.468456] [G loss: -49.181351]
[Epoch 0/200] [Batch 10/815] [D loss: -3.809268] [G loss: -37.151054]
[Epoch 0/200] [Batch 15/815] [D loss: -3.958514] [G loss: -44.932156]
[Epoch 0/200] [Batch 20/815] [D loss: -4.714270] [G loss: -30.208340]
[Epoch 0/200] [Batch 25/815] [D loss: -5.454857] [G loss: -22.341024]
[Epoch 0/200] [Batch 30/815] [D loss: -4.707932] [G loss: -34.585598]
[Epoch 0/200] [Batch 35/815] [D loss: -5.001633] [G loss: -23.999882]
[Epoch 0/200] [Batch 40/815] [D loss: -4.780243] [G loss: -37.460815]
[Epoch 0/200] [Batch 45/815] [D loss: -4.962447] [G loss: -31.616678]
[Epoch 0/200] [Batch 50/815] [D loss: -5.444920] [G loss: -22.370380]
[Epoch 0/200] [Batch 55/815] [D loss: -5.580760] [G loss: -20.079750]
[Epoch 0/200] [Batch 60/815] [D loss: -4.717446] [G loss: -28.510597]
[Epoch 0/200] [Batch 65/815] [D loss: -5.193073] [G loss: -21.988544]
[Epoch 0/200] [Batch 7

[Epoch 0/200] [Batch 585/815] [D loss: -5.223890] [G loss: -18.083355]
[Epoch 0/200] [Batch 590/815] [D loss: -5.354591] [G loss: -13.752713]
[Epoch 0/200] [Batch 595/815] [D loss: -4.550429] [G loss: -25.927073]
[Epoch 0/200] [Batch 600/815] [D loss: -5.660472] [G loss: -18.913204]
[Epoch 0/200] [Batch 605/815] [D loss: -5.237696] [G loss: -16.364305]
[Epoch 0/200] [Batch 610/815] [D loss: -4.806082] [G loss: -26.446293]
[Epoch 0/200] [Batch 615/815] [D loss: -5.402307] [G loss: -16.312006]
[Epoch 0/200] [Batch 620/815] [D loss: -3.989680] [G loss: -34.518536]
[Epoch 0/200] [Batch 625/815] [D loss: -4.385514] [G loss: -19.857111]
[Epoch 0/200] [Batch 630/815] [D loss: -4.287412] [G loss: -19.260242]
[Epoch 0/200] [Batch 635/815] [D loss: -5.114437] [G loss: -12.806242]
[Epoch 0/200] [Batch 640/815] [D loss: -4.560164] [G loss: -23.153900]
[Epoch 0/200] [Batch 645/815] [D loss: -4.728593] [G loss: -15.868505]
[Epoch 0/200] [Batch 650/815] [D loss: -3.840821] [G loss: -36.611881]
[Epoch

[Epoch 1/200] [Batch 350/815] [D loss: -4.492887] [G loss: -25.279736]
[Epoch 1/200] [Batch 355/815] [D loss: -3.738382] [G loss: -24.328417]
[Epoch 1/200] [Batch 360/815] [D loss: -4.610808] [G loss: -20.882832]
[Epoch 1/200] [Batch 365/815] [D loss: -3.713732] [G loss: -31.430363]
[Epoch 1/200] [Batch 370/815] [D loss: -5.524736] [G loss: -23.077261]
[Epoch 1/200] [Batch 375/815] [D loss: -4.979402] [G loss: -23.893024]
[Epoch 1/200] [Batch 380/815] [D loss: -4.843578] [G loss: -32.957684]
[Epoch 1/200] [Batch 385/815] [D loss: -4.058051] [G loss: -23.604017]
[Epoch 1/200] [Batch 390/815] [D loss: -5.049287] [G loss: -21.154758]
[Epoch 1/200] [Batch 395/815] [D loss: -4.873770] [G loss: -23.605249]
[Epoch 1/200] [Batch 400/815] [D loss: -5.215153] [G loss: -20.794115]
[Epoch 1/200] [Batch 405/815] [D loss: -5.262623] [G loss: -28.631025]
[Epoch 1/200] [Batch 410/815] [D loss: -4.094130] [G loss: -30.661709]
[Epoch 1/200] [Batch 415/815] [D loss: -4.576552] [G loss: -20.956545]
[Epoch

[Epoch 2/200] [Batch 120/815] [D loss: -6.181818] [G loss: -15.993863]
[Epoch 2/200] [Batch 125/815] [D loss: -5.168097] [G loss: -16.417250]
[Epoch 2/200] [Batch 130/815] [D loss: -5.836532] [G loss: -14.296454]
[Epoch 2/200] [Batch 135/815] [D loss: -6.008942] [G loss: -14.191665]
[Epoch 2/200] [Batch 140/815] [D loss: -5.841182] [G loss: -23.600376]
[Epoch 2/200] [Batch 145/815] [D loss: -5.564143] [G loss: -16.364117]
[Epoch 2/200] [Batch 150/815] [D loss: -5.586823] [G loss: -15.999202]
[Epoch 2/200] [Batch 155/815] [D loss: -5.794216] [G loss: -16.602539]
[Epoch 2/200] [Batch 160/815] [D loss: -5.515720] [G loss: -21.756300]
[Epoch 2/200] [Batch 165/815] [D loss: -5.373440] [G loss: -16.167997]
[Epoch 2/200] [Batch 170/815] [D loss: -4.347470] [G loss: -28.155409]
[Epoch 2/200] [Batch 175/815] [D loss: -4.487490] [G loss: -23.627829]
[Epoch 2/200] [Batch 180/815] [D loss: -4.811705] [G loss: -17.013186]
[Epoch 2/200] [Batch 185/815] [D loss: -4.126747] [G loss: -20.725803]
[Epoch

[Epoch 2/200] [Batch 700/815] [D loss: -4.263895] [G loss: -34.927174]
[Epoch 2/200] [Batch 705/815] [D loss: -4.491981] [G loss: -27.283791]
[Epoch 2/200] [Batch 710/815] [D loss: -4.582393] [G loss: -27.769751]
[Epoch 2/200] [Batch 715/815] [D loss: -4.691870] [G loss: -18.148865]
[Epoch 2/200] [Batch 720/815] [D loss: -4.580259] [G loss: -20.294664]
[Epoch 2/200] [Batch 725/815] [D loss: -5.083668] [G loss: -17.739964]
[Epoch 2/200] [Batch 730/815] [D loss: -6.142824] [G loss: -19.871464]
[Epoch 2/200] [Batch 735/815] [D loss: -4.642027] [G loss: -28.547817]
[Epoch 2/200] [Batch 740/815] [D loss: -5.725635] [G loss: -18.805687]
[Epoch 2/200] [Batch 745/815] [D loss: -4.696927] [G loss: -19.385223]
[Epoch 2/200] [Batch 750/815] [D loss: -4.713256] [G loss: -16.629835]
[Epoch 2/200] [Batch 755/815] [D loss: -5.537938] [G loss: -16.493715]
[Epoch 2/200] [Batch 760/815] [D loss: -5.291945] [G loss: -20.046896]
[Epoch 2/200] [Batch 765/815] [D loss: -4.684738] [G loss: -15.802648]
[Epoch

...

[Epoch 22/200] [Batch 560/815] [D loss: -4.792502] [G loss: -17.777403]
[Epoch 22/200] [Batch 565/815] [D loss: -5.357608] [G loss: -17.736792]
[Epoch 22/200] [Batch 570/815] [D loss: -5.679651] [G loss: -21.296736]
[Epoch 22/200] [Batch 575/815] [D loss: -5.548481] [G loss: -10.936157]
[Epoch 22/200] [Batch 580/815] [D loss: -4.550591] [G loss: -27.855610]
[Epoch 22/200] [Batch 585/815] [D loss: -4.523858] [G loss: -18.312325]
[Epoch 22/200] [Batch 590/815] [D loss: -5.114099] [G loss: -15.784842]
[Epoch 22/200] [Batch 595/815] [D loss: -6.271438] [G loss: -13.601686]
[Epoch 22/200] [Batch 600/815] [D loss: -4.951185] [G loss: -15.485799]
[Epoch 22/200] [Batch 605/815] [D loss: -5.708110] [G loss: -12.852815]
[Epoch 22/200] [Batch 610/815] [D loss: -4.640228] [G loss: -16.716490]
[Epoch 22/200] [Batch 615/815] [D loss: -4.290450] [G loss: -25.691175]
[Epoch 22/200] [Batch 620/815] [D loss: -4.590927] [G loss: -18.197966]
[Epoch 22/200] [Batch 625/815] [D loss: -5.297795] [G loss: -13.

[Epoch 23/200] [Batch 320/815] [D loss: -5.465219] [G loss: -14.496416]
[Epoch 23/200] [Batch 325/815] [D loss: -5.260272] [G loss: -25.691799]
[Epoch 23/200] [Batch 330/815] [D loss: -5.027196] [G loss: -15.414821]
[Epoch 23/200] [Batch 335/815] [D loss: -5.416794] [G loss: -14.018203]
[Epoch 23/200] [Batch 340/815] [D loss: -7.040467] [G loss: -14.527547]
[Epoch 23/200] [Batch 345/815] [D loss: -6.194490] [G loss: -16.032089]
[Epoch 23/200] [Batch 350/815] [D loss: -5.040563] [G loss: -20.084900]
[Epoch 23/200] [Batch 355/815] [D loss: -4.643407] [G loss: -27.123898]
[Epoch 23/200] [Batch 360/815] [D loss: -5.685476] [G loss: -15.525868]
[Epoch 23/200] [Batch 365/815] [D loss: -5.331993] [G loss: -15.218394]
[Epoch 23/200] [Batch 370/815] [D loss: -4.217785] [G loss: -27.734028]
[Epoch 23/200] [Batch 375/815] [D loss: -6.104823] [G loss: -12.909054]
[Epoch 23/200] [Batch 380/815] [D loss: -5.677057] [G loss: -18.931154]
[Epoch 23/200] [Batch 385/815] [D loss: -6.032331] [G loss: -16.

[Epoch 24/200] [Batch 80/815] [D loss: -5.736144] [G loss: -13.674162]
[Epoch 24/200] [Batch 85/815] [D loss: -5.668692] [G loss: -15.417385]
[Epoch 24/200] [Batch 90/815] [D loss: -5.491690] [G loss: -18.679838]
[Epoch 24/200] [Batch 95/815] [D loss: -5.378197] [G loss: -20.123749]
[Epoch 24/200] [Batch 100/815] [D loss: -5.262973] [G loss: -20.344555]
[Epoch 24/200] [Batch 105/815] [D loss: -6.391011] [G loss: -17.259932]
[Epoch 24/200] [Batch 110/815] [D loss: -6.459971] [G loss: -12.969234]
[Epoch 24/200] [Batch 115/815] [D loss: -3.870505] [G loss: -24.292603]
[Epoch 24/200] [Batch 120/815] [D loss: -6.171345] [G loss: -13.590130]
[Epoch 24/200] [Batch 125/815] [D loss: -5.966655] [G loss: -15.932622]
[Epoch 24/200] [Batch 130/815] [D loss: -4.898588] [G loss: -29.784763]
[Epoch 24/200] [Batch 135/815] [D loss: -5.514517] [G loss: -17.325377]
[Epoch 24/200] [Batch 140/815] [D loss: -5.652320] [G loss: -16.724350]
[Epoch 24/200] [Batch 145/815] [D loss: -5.260108] [G loss: -14.0434

[Epoch 24/200] [Batch 650/815] [D loss: -6.122011] [G loss: -13.672697]
[Epoch 24/200] [Batch 655/815] [D loss: -4.573861] [G loss: -22.845694]
[Epoch 24/200] [Batch 660/815] [D loss: -5.392619] [G loss: -15.626405]
[Epoch 24/200] [Batch 665/815] [D loss: -5.233712] [G loss: -21.129736]
[Epoch 24/200] [Batch 670/815] [D loss: -5.842145] [G loss: -12.624260]
[Epoch 24/200] [Batch 675/815] [D loss: -5.822451] [G loss: -15.162377]
[Epoch 24/200] [Batch 680/815] [D loss: -5.028018] [G loss: -14.488060]
[Epoch 24/200] [Batch 685/815] [D loss: -5.695107] [G loss: -15.559119]
[Epoch 24/200] [Batch 690/815] [D loss: -5.496065] [G loss: -16.409756]
[Epoch 24/200] [Batch 695/815] [D loss: -5.843168] [G loss: -15.111381]
[Epoch 24/200] [Batch 700/815] [D loss: -5.486088] [G loss: -12.363657]
[Epoch 24/200] [Batch 705/815] [D loss: -5.332354] [G loss: -23.805101]
[Epoch 24/200] [Batch 710/815] [D loss: -5.073395] [G loss: -16.117563]
[Epoch 24/200] [Batch 715/815] [D loss: -6.138811] [G loss: -15.

OSError: [Errno 12] Cannot allocate memory

In [57]:
generator.state_dict()

OrderedDict([('model.0.weight',
              tensor([[-0.0755,  0.0834, -0.0273,  ..., -0.0811, -0.0297,  0.0153],
                      [ 0.1134,  0.0469,  0.0029,  ..., -0.0919,  0.0523,  0.0084],
                      [-0.0145,  0.0043, -0.1151,  ..., -0.0460, -0.0711, -0.0073],
                      ...,
                      [-0.0289,  0.0349, -0.0487,  ..., -0.0206,  0.0813,  0.0843],
                      [-0.0824,  0.0009,  0.0197,  ...,  0.0484, -0.0272, -0.0106],
                      [-0.0122,  0.0017, -0.1745,  ..., -0.0262, -0.0026,  0.0457]],
                     device='cuda:0')),
             ('model.0.bias',
              tensor([ 0.0516, -0.0204,  0.2660,  0.1415,  0.0396,  0.0179, -0.0682,  0.1631,
                       0.4934,  0.1693,  0.1395,  0.2470,  0.2450,  0.1272, -0.2084, -0.0913,
                       0.1224,  0.2447,  0.0536,  0.1853,  0.1298,  0.3033,  0.1689,  0.1224,
                      -0.7369,  0.0848,  0.4795,  0.4657,  0.1758,  0.2944,  0.3005,

### Save the models

In [76]:
os.makedirs("/home/jgmeyer2/vangan/gans/models",exist_ok=True)
PATH = "/home/jgmeyer2/vangan/gans/models/g5k.model"
modelid="5k"


state_g = {
    'epoch': epoch,
    'state_dict': generator.state_dict(),
    'optimizer': optimizer_G.state_dict()
    }
torch.save(state_g, PATH+"g"+modelid+".model")

state_d = {
    'epoch': epoch,
    'state_dict': discriminator.state_dict(),
    'optimizer': optimizer_D.state_dict()
    }
torch.save(state_d, PATH+"d"+modelid+".model")
print("saved models @")
print(epoch)


#def save_model(net, optim, ckpt_fname):
#    state_dict = net.module.state_dict()
#    for key in state_dict.keys():
#        state_dict[key] = state_dict[key].cpu()
#        torch.save({
#            'epoch': epoch,                                                                                                                                                                                     
#            'state_dict': state_dict,                                                                                                                                                                                
#            'optimizer': optim},                                                                                                                                                                                     
#            ckpt_fname)

saved models @
4999


# Load models

In [ ]:



model.load_state_dict(state['state_dict'])
optimizer.load_state_dict(state['optimizer'])

model.load_state_dict(state['state_dict'])
optimizer.load_state_dict(state['optimizer'])

state = torch.load(filepath)

